In [6]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
import gc

# Set memory optimization flags
torch.backends.cudnn.benchmark = True
torch.cuda.empty_cache()

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Create a custom dataset for multimodal data
class MultimodalDataset(Dataset):
    def __init__(self, emg_data, eeg_data, labels):
        self.emg_data = torch.FloatTensor(emg_data).to(device)
        self.eeg_data = torch.FloatTensor(eeg_data).to(device)
        self.labels = torch.LongTensor(labels).to(device)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.emg_data[idx], self.eeg_data[idx], self.labels[idx]

# Define the EMG encoder network
class EMGEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(EMGEncoder, self).__init__()
        self.conv1 = nn.Conv1d(input_dim, hidden_dim, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, x):
        # x shape: [batch_size, sequence_length, channels]
        x = x.permute(0, 2, 1)  # [batch_size, channels, sequence_length]
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = self.dropout(x)
        # Global average pooling
        x = torch.mean(x, dim=2)
        return x

# Define the EEG encoder network
class EEGEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(EEGEncoder, self).__init__()
        self.conv1 = nn.Conv1d(input_dim, hidden_dim, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, x):
        # x shape: [batch_size, sequence_length, channels]
        x = x.permute(0, 2, 1)  # [batch_size, channels, sequence_length]
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = self.dropout(x)
        # Global average pooling
        x = torch.mean(x, dim=2)
        return x

# Define the multimodal fusion network
class MultimodalNet(nn.Module):
    def __init__(self, emg_input_dim, eeg_input_dim, hidden_dim, window_size, num_classes):
        super(MultimodalNet, self).__init__()
        
        self.emg_encoder = EMGEncoder(emg_input_dim, hidden_dim)
        self.eeg_encoder = EEGEncoder(eeg_input_dim, hidden_dim)
        
        # Fusion layer
        self.fusion = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_dim, num_classes)
        )
        
    def forward(self, emg, eeg):
        emg_features = self.emg_encoder(emg)
        eeg_features = self.eeg_encoder(eeg)
        
        # Concatenate features
        combined = torch.cat((emg_features, eeg_features), dim=1)
        
        # Pass through fusion layer
        output = self.fusion(combined)
        return output

# Process and load the data
def load_and_process_data(emg_path, eeg_path, window_size=50, stride=25):
    print("Loading data...")
    
    # Load data
    emg_data = pd.read_csv(emg_path)
    eeg_data = pd.read_csv(eeg_path)
    
    # Extract features and labels
    emg_features = emg_data.iloc[:, :8].values
    eeg_features = eeg_data.iloc[:, :8].values
    
    # Standardize the features
    print("Normalizing data...")
    emg_scaler = StandardScaler()
    eeg_scaler = StandardScaler()
    
    emg_features = emg_scaler.fit_transform(emg_features)
    eeg_features = eeg_scaler.fit_transform(eeg_features)
    
    # Create windowed data
    emg_windows = []
    eeg_windows = []
    window_labels = []
    
    # Find common samples between EMG and EEG data
    emg_samples = set(tuple(x) for x in emg_data[['subject', 'repetition', 'gesture']].drop_duplicates().values)
    eeg_samples = set(tuple(x) for x in eeg_data[['subject', 'repetition', 'gesture']].drop_duplicates().values)
    common_samples = emg_samples.intersection(eeg_samples)
    
    print(f"Found {len(common_samples)} common samples between EMG and EEG data.")
    
    for sample in common_samples:
        subject, repetition, gesture = sample
        
        # Get data for this sample
        emg_sample = emg_data[(emg_data['subject'] == subject) & 
                              (emg_data['repetition'] == repetition) & 
                              (emg_data['gesture'] == gesture)]
        
        eeg_sample = eeg_data[(eeg_data['subject'] == subject) & 
                              (eeg_data['repetition'] == repetition) & 
                              (eeg_data['gesture'] == gesture)]
        
        # Make sure both samples have data
        if len(emg_sample) == 0 or len(eeg_sample) == 0:
            continue
            
        # Extract features
        emg_sample_features = emg_sample.iloc[:, :8].values
        eeg_sample_features = eeg_sample.iloc[:, :8].values
        
        # Standardize using pre-fitted scalers
        emg_sample_features = emg_scaler.transform(emg_sample_features)
        eeg_sample_features = eeg_scaler.transform(eeg_sample_features)
        
        # Handle different lengths by using the shorter one
        min_length = min(len(emg_sample_features), len(eeg_sample_features))
        if min_length <= window_size:
            continue  # Skip if sample is too short
            
        emg_sample_features = emg_sample_features[:min_length]
        eeg_sample_features = eeg_sample_features[:min_length]
        
        # Create windows
        for i in range(0, min_length - window_size, stride):
            emg_windows.append(emg_sample_features[i:i+window_size])
            eeg_windows.append(eeg_sample_features[i:i+window_size])
            window_labels.append(gesture - 1)  # 0-indexed labels
    
    if len(emg_windows) == 0:
        raise ValueError("No valid windows could be created. Check your data alignment.")
    
    print(f"Created {len(emg_windows)} windows from {len(common_samples)} samples.")
    
    return np.array(emg_windows), np.array(eeg_windows), np.array(window_labels)

# Training function with memory optimizations
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for i, (emg_inputs, eeg_inputs, labels) in enumerate(train_loader):
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(emg_inputs, eeg_inputs)
            loss = criterion(outputs, labels)
            
            # Backward and optimize
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Free up memory
            if i % 10 == 0:
                torch.cuda.empty_cache()
        
        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for emg_inputs, eeg_inputs, labels in val_loader:
                outputs = model(emg_inputs, eeg_inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
                # Free memory
                del outputs, loss
        
        torch.cuda.empty_cache()
        val_loss = val_loss / len(val_loader)
        val_acc = 100 * correct / total
        
        print(f'Epoch {epoch+1}/{num_epochs}, '
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
        
        # Save the best model
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), 'best_multimodal_model.pth')
            print(f'Saved model with accuracy: {val_acc:.2f}%')
    
    return model

def main():
    # Hyperparameters
    window_size = 50
    batch_size = 16  # Smaller batch size to save memory
    learning_rate = 0.001
    num_epochs = 30
    hidden_dim = 64  # Reduced hidden dimensions
    
    # Set paths to your data files
    emg_path = 'data/processed/EMG-data.csv'
    eeg_path = 'data/processed/EEG-data.csv'
    
    try:
        # Load and process data
        emg_windows, eeg_windows, window_labels = load_and_process_data(
            emg_path, eeg_path, window_size=window_size
        )
        
        # Calculate number of unique classes
        num_classes = len(np.unique(window_labels))
        print(f"Number of classes: {num_classes}")
        print(f"EMG windows shape: {emg_windows.shape}")
        print(f"EEG windows shape: {eeg_windows.shape}")
        
        # Split the data into training and testing sets
        X_emg_train, X_emg_test, X_eeg_train, X_eeg_test, y_train, y_test = train_test_split(
            emg_windows, eeg_windows, window_labels, test_size=0.2, random_state=42, stratify=window_labels
        )
        
        # Free up memory
        del emg_windows, eeg_windows, window_labels
        gc.collect()
        torch.cuda.empty_cache()
        
        # Create datasets
        train_dataset = MultimodalDataset(X_emg_train, X_eeg_train, y_train)
        test_dataset = MultimodalDataset(X_emg_test, X_eeg_test, y_test)
        
        # Free up more memory
        del X_emg_train, X_emg_test, X_eeg_train, X_eeg_test, y_train, y_test
        gc.collect()
        torch.cuda.empty_cache()
        
        # Create data loaders with smaller batch sizes
        train_loader = DataLoader(
            train_dataset, batch_size=batch_size, shuffle=True, 
            pin_memory=False  # Disable pin_memory to save RAM
        )
        
        test_loader = DataLoader(
            test_dataset, batch_size=batch_size, shuffle=False,
            pin_memory=False
        )
        
        # Initialize model
        emg_input_dim = 8  # Number of EMG channels
        eeg_input_dim = 8  # Number of EEG channels
        
        model = MultimodalNet(
            emg_input_dim=emg_input_dim,
            eeg_input_dim=eeg_input_dim,
            hidden_dim=hidden_dim,
            window_size=window_size,
            num_classes=num_classes
        ).to(device)
        
        # Define optimizer and loss function
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
        
        # Train the model
        trained_model = train_model(
            model, train_loader, test_loader, criterion, optimizer, num_epochs=num_epochs
        )
        
        print("Training completed!")
        
        # Evaluate the model
        trained_model.eval()
        correct = 0
        total = 0
        
        with torch.no_grad():
            for emg_inputs, eeg_inputs, labels in test_loader:
                outputs = trained_model(emg_inputs, eeg_inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        print(f'Test Accuracy: {100 * correct / total:.2f}%')
        
    except Exception as e:
        print(f"Error occurred: {e}")
        print("\nAlternative approach: Let's try with paired data synchronization...")
        # If the first method fails, you could implement an alternative approach here

if __name__ == "__main__":
    main()

Using device: cuda
Loading data...
Normalizing data...
Found 378 common samples between EMG and EEG data.
Created 19576 windows from 378 samples.
Number of classes: 7
EMG windows shape: (19576, 50, 8)
EEG windows shape: (19576, 50, 8)
Epoch 1/30, Train Loss: 1.2394, Train Acc: 52.67%, Val Loss: 0.7855, Val Acc: 74.97%
Saved model with accuracy: 74.97%
Epoch 2/30, Train Loss: 0.7856, Train Acc: 70.62%, Val Loss: 0.5693, Val Acc: 83.12%
Saved model with accuracy: 83.12%
Epoch 3/30, Train Loss: 0.6459, Train Acc: 76.80%, Val Loss: 0.4472, Val Acc: 86.87%
Saved model with accuracy: 86.87%
Epoch 4/30, Train Loss: 0.5508, Train Acc: 80.02%, Val Loss: 0.3651, Val Acc: 89.81%
Saved model with accuracy: 89.81%
Epoch 5/30, Train Loss: 0.4818, Train Acc: 83.03%, Val Loss: 0.3118, Val Acc: 90.96%
Saved model with accuracy: 90.96%
Epoch 6/30, Train Loss: 0.4345, Train Acc: 84.74%, Val Loss: 0.2437, Val Acc: 94.41%
Saved model with accuracy: 94.41%
Epoch 7/30, Train Loss: 0.3887, Train Acc: 86.40%, 

In [11]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.amp import autocast, GradScaler
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
import gc
from torch.cuda.amp import autocast, GradScaler
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import random
from torch.optim.lr_scheduler import CosineAnnealingLR

# Set memory optimization flags and reproducibility
torch.backends.cudnn.benchmark = True
torch.cuda.empty_cache()
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Create a custom dataset for multimodal data - Fixed to load data to GPU only when needed
class MultimodalDataset(Dataset):
    def __init__(self, emg_data, eeg_data, labels):
        self.emg_data = torch.FloatTensor(emg_data)
        self.eeg_data = torch.FloatTensor(eeg_data)
        self.labels = torch.LongTensor(labels)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.emg_data[idx], self.eeg_data[idx], self.labels[idx]

# Define the improved EMG encoder network
class EMGEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(EMGEncoder, self).__init__()
        # Deeper architecture with multiple convolutional layers
        self.conv1 = nn.Conv1d(input_dim, hidden_dim, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.conv2 = nn.Conv1d(hidden_dim, hidden_dim*2, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(hidden_dim*2)
        self.conv3 = nn.Conv1d(hidden_dim*2, hidden_dim*2, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(hidden_dim*2)
        self.pool = nn.MaxPool1d(2)
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, x):
        # x shape: [batch_size, sequence_length, channels]
        x = x.permute(0, 2, 1)  # [batch_size, channels, sequence_length]
        
        # First convolutional block
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dropout(x)
        
        # Second convolutional block
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dropout(x)
        
        # Third convolutional block
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        
        # Both global average pooling and max pooling for better feature extraction
        avg_pool = torch.mean(x, dim=2)
        max_pool, _ = torch.max(x, dim=2)
        
        # Concatenate both pooling results
        x = torch.cat([avg_pool, max_pool], dim=1)
        return x

# Define the improved EEG encoder network
class EEGEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(EEGEncoder, self).__init__()
        # Deeper architecture with multiple convolutional layers
        self.conv1 = nn.Conv1d(input_dim, hidden_dim, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.conv2 = nn.Conv1d(hidden_dim, hidden_dim*2, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(hidden_dim*2)
        self.conv3 = nn.Conv1d(hidden_dim*2, hidden_dim*2, kernel_size=3, padding=1) 
        self.bn3 = nn.BatchNorm1d(hidden_dim*2)
        self.pool = nn.MaxPool1d(2)
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, x):
        # x shape: [batch_size, sequence_length, channels]
        x = x.permute(0, 2, 1)  # [batch_size, channels, sequence_length]
        
        # First convolutional block
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dropout(x)
        
        # Second convolutional block
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dropout(x)
        
        # Third convolutional block
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        
        # Both global average pooling and max pooling for better feature extraction
        avg_pool = torch.mean(x, dim=2)
        max_pool, _ = torch.max(x, dim=2)
        
        # Concatenate both pooling results
        x = torch.cat([avg_pool, max_pool], dim=1)
        return x

# Define the improved multimodal fusion network
class MultimodalNet(nn.Module):
    def __init__(self, emg_input_dim, eeg_input_dim, hidden_dim, num_classes):
        super(MultimodalNet, self).__init__()
        
        self.emg_encoder = EMGEncoder(emg_input_dim, hidden_dim)
        self.eeg_encoder = EEGEncoder(eeg_input_dim, hidden_dim)
        
        # Calculate the size of concatenated features (doubled due to dual pooling)
        concat_size = hidden_dim * 2 * 2 * 2  # 2 encoders x 2 feature sizes each x 2 pooling types
        
        # Improved fusion layer with additional layers
        self.fusion = nn.Sequential(
            nn.Linear(concat_size, hidden_dim * 4),
            nn.BatchNorm1d(hidden_dim * 4),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_dim * 4, hidden_dim * 2),
            nn.BatchNorm1d(hidden_dim * 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim * 2, num_classes)
        )
        
    def forward(self, emg, eeg):
        emg_features = self.emg_encoder(emg)
        eeg_features = self.eeg_encoder(eeg)
        
        # Concatenate features
        combined = torch.cat((emg_features, eeg_features), dim=1)
        
        # Pass through fusion layer
        output = self.fusion(combined)
        return output

# Add noise to data for augmentation
def add_noise(data, noise_factor=0.05):
    noise = torch.randn(data.shape).to(data.device) * noise_factor * torch.std(data)
    return data + noise

# Process and load the data
def load_and_process_data(emg_path, eeg_path, window_size=50, stride=25):
    print("Loading data...")
    
    # Load data
    emg_data = pd.read_csv(emg_path)
    eeg_data = pd.read_csv(eeg_path)
    
    # Extract features and labels
    emg_features = emg_data.iloc[:, :8].values
    eeg_features = eeg_data.iloc[:, :8].values
    
    # Standardize the features
    print("Normalizing data...")
    emg_scaler = StandardScaler()
    eeg_scaler = StandardScaler()
    
    emg_features = emg_scaler.fit_transform(emg_features)
    eeg_features = eeg_scaler.fit_transform(eeg_features)
    
    # Create windowed data
    emg_windows = []
    eeg_windows = []
    window_labels = []
    sample_ids = []  # Track sample IDs for stratified splits
    
    # Find common samples between EMG and EEG data
    emg_samples = set(tuple(x) for x in emg_data[['subject', 'repetition', 'gesture']].drop_duplicates().values)
    eeg_samples = set(tuple(x) for x in eeg_data[['subject', 'repetition', 'gesture']].drop_duplicates().values)
    common_samples = emg_samples.intersection(eeg_samples)
    
    print(f"Found {len(common_samples)} common samples between EMG and EEG data.")
    
    for sample in common_samples:
        subject, repetition, gesture = sample
        
        # Get data for this sample
        emg_sample = emg_data[(emg_data['subject'] == subject) & 
                              (emg_data['repetition'] == repetition) & 
                              (emg_data['gesture'] == gesture)]
        
        eeg_sample = eeg_data[(eeg_data['subject'] == subject) & 
                              (eeg_data['repetition'] == repetition) & 
                              (eeg_data['gesture'] == gesture)]
        
        # Make sure both samples have data
        if len(emg_sample) == 0 or len(eeg_sample) == 0:
            continue
            
        # Extract features
        emg_sample_features = emg_sample.iloc[:, :8].values
        eeg_sample_features = eeg_sample.iloc[:, :8].values
        
        # Standardize using pre-fitted scalers
        emg_sample_features = emg_scaler.transform(emg_sample_features)
        eeg_sample_features = eeg_scaler.transform(eeg_sample_features)
        
        # Handle different lengths by using the shorter one
        min_length = min(len(emg_sample_features), len(eeg_sample_features))
        if min_length <= window_size:
            continue  # Skip if sample is too short
            
        emg_sample_features = emg_sample_features[:min_length]
        eeg_sample_features = eeg_sample_features[:min_length]
        
        # Create windows
        for i in range(0, min_length - window_size, stride):
            emg_windows.append(emg_sample_features[i:i+window_size])
            eeg_windows.append(eeg_sample_features[i:i+window_size])
            window_labels.append(gesture - 1)  # 0-indexed labels
            sample_ids.append(f"{subject}_{repetition}_{gesture}")  # Track which sample this comes from
    
    if len(emg_windows) == 0:
        raise ValueError("No valid windows could be created. Check your data alignment.")
    
    print(f"Created {len(emg_windows)} windows from {len(common_samples)} samples.")
    
    return np.array(emg_windows), np.array(eeg_windows), np.array(window_labels), np.array(sample_ids)

# Training function with memory optimizations, mixed precision, and metrics tracking
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10):
    best_acc = 0.0
    best_f1 = 0.0
    
    # Initialize gradient scaler for mixed precision training - FIXED
    scaler = GradScaler()
    
    # Keep track of metrics
    history = {
        'train_loss': [], 'train_acc': [], 'train_f1': [],
        'val_loss': [], 'val_acc': [], 'val_f1': [],
    }
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        all_preds = []
        all_targets = []
        
        for i, (emg_inputs, eeg_inputs, labels) in enumerate(train_loader):
            # Move data to device
            emg_inputs, eeg_inputs, labels = emg_inputs.to(device), eeg_inputs.to(device), labels.to(device)
            
            # Data augmentation (add noise) in training only
            if random.random() < 0.5:  # 50% chance to apply noise
                emg_inputs = add_noise(emg_inputs, noise_factor=0.03)
                eeg_inputs = add_noise(eeg_inputs, noise_factor=0.03)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass with mixed precision - FIXED
            with autocast():
                outputs = model(emg_inputs, eeg_inputs)
                loss = criterion(outputs, labels)
            
            # Backward and optimize with gradient scaling
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            
            # Store predictions and labels for F1 score calculation
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())
            
            # Free up memory
            if i % 10 == 0:
                torch.cuda.empty_cache()
        
        # Update learning rate
        scheduler.step()
        
        # Calculate epoch metrics
        train_loss = running_loss / len(train_loader)
        train_acc = 100 * np.mean(np.array(all_preds) == np.array(all_targets))
        train_f1 = f1_score(all_targets, all_preds, average='weighted')
        
        # Rest of the function remains the same...
               
        # Store metrics
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['train_f1'].append(train_f1)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_targets = []
        
        with torch.no_grad():
            for emg_inputs, eeg_inputs, labels in val_loader:
                # Move data to device
                emg_inputs, eeg_inputs, labels = emg_inputs.to(device), eeg_inputs.to(device), labels.to(device)
                
                # Forward pass (no mixed precision needed for validation)
                outputs = model(emg_inputs, eeg_inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                
                # Store predictions and labels
                all_preds.extend(predicted.cpu().numpy())
                all_targets.extend(labels.cpu().numpy())
                
                # Free memory
                del outputs, loss
        
        torch.cuda.empty_cache()
        val_loss = val_loss / len(val_loader)
        val_acc = 100 * np.mean(np.array(all_preds) == np.array(all_targets))
        val_f1 = f1_score(all_targets, all_preds, average='weighted')
        val_precision = precision_score(all_targets, all_preds, average='weighted')
        val_recall = recall_score(all_targets, all_preds, average='weighted')
        
        # Store metrics
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['val_f1'].append(val_f1)
        
        print(f'Epoch {epoch+1}/{num_epochs}, '
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Train F1: {train_f1:.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%, Val F1: {val_f1:.4f}')
        print(f'Precision: {val_precision:.4f}, Recall: {val_recall:.4f}')
        
        # Save the best model based on F1 score (more informative than accuracy)
        # In the train_model function, update the model saving part:
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_acc = val_acc
            # Save only the necessary components
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'val_f1': val_f1,
                'val_acc': val_acc,
            }, 'best_multimodal_model.pth')
            # Save history separately if needed
            np.save('training_history.npy', history)
            print(f'Saved model with F1: {val_f1:.4f}, Accuracy: {val_acc:.2f}%')
    
    return model, history
# Function to evaluate on test set
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for emg_inputs, eeg_inputs, labels in test_loader:
            # Move data to device
            emg_inputs, eeg_inputs, labels = emg_inputs.to(device), eeg_inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(emg_inputs, eeg_inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            # Store predictions and labels
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())
    
    # Calculate metrics
    accuracy = 100 * np.mean(np.array(all_preds) == np.array(all_targets))
    f1 = f1_score(all_targets, all_preds, average='weighted')
    precision = precision_score(all_targets, all_preds, average='weighted')
    recall = recall_score(all_targets, all_preds, average='weighted')
    conf_matrix = confusion_matrix(all_targets, all_preds)
    
    print("Test Results:")
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print("Confusion Matrix:")
    print(conf_matrix)
    
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'confusion_matrix': conf_matrix
    }

def main():
    # Make sure numpy is imported inside the function 
    import numpy as np
    import torch
    import gc
    from sklearn.model_selection import train_test_split
    
    # Hyperparameters
    window_size = 50
    batch_size = 32  # Adjusted batch size for mixed precision
    learning_rate = 0.001
    num_epochs = 30
    hidden_dim = 64
    
    # Set paths to your data files
    emg_path = 'data/processed/EMG-data.csv'
    eeg_path = 'data/processed/EEG-data.csv'
    
    try:
        # Load and process data
        emg_windows, eeg_windows, window_labels, sample_ids = load_and_process_data(
            emg_path, eeg_path, window_size=window_size
        )
        
        # Calculate number of unique classes
        num_classes = len(np.unique(window_labels))
        print(f"Number of classes: {num_classes}")
        print(f"EMG windows shape: {emg_windows.shape}")
        print(f"EEG windows shape: {eeg_windows.shape}")
        
        # New: Proper train/val/test split (60/20/20) - stratified by sample_id to prevent data leakage
        # First, get unique sample_ids
        unique_samples = np.unique(sample_ids)
        
        # Use sklearn's train_test_split to split the sample IDs
        samples_train, samples_temp = train_test_split(
            unique_samples, test_size=0.4, random_state=42
        )
        samples_val, samples_test = train_test_split(
            samples_temp, test_size=0.5, random_state=42
        )
        
        # Now create masks based on these sample IDs
        train_mask = np.isin(sample_ids, samples_train)
        val_mask = np.isin(sample_ids, samples_val)
        test_mask = np.isin(sample_ids, samples_test)
        
        # Apply masks to get the actual data splits
        X_emg_train, X_eeg_train, y_train = emg_windows[train_mask], eeg_windows[train_mask], window_labels[train_mask]
        X_emg_val, X_eeg_val, y_val = emg_windows[val_mask], eeg_windows[val_mask], window_labels[val_mask]
        X_emg_test, X_eeg_test, y_test = emg_windows[test_mask], eeg_windows[test_mask], window_labels[test_mask]
        
        print(f"Training set: {len(X_emg_train)} samples")
        print(f"Validation set: {len(X_emg_val)} samples")
        print(f"Test set: {len(X_emg_test)} samples")
        
        # Free up memory
        del emg_windows, eeg_windows, window_labels, sample_ids
        gc.collect()
        torch.cuda.empty_cache()
        
        # Create datasets
        train_dataset = MultimodalDataset(X_emg_train, X_eeg_train, y_train)
        val_dataset = MultimodalDataset(X_emg_val, X_eeg_val, y_val)
        test_dataset = MultimodalDataset(X_emg_test, X_eeg_test, y_test)
        
        # Free up more memory
        del X_emg_train, X_emg_val, X_emg_test, X_eeg_train, X_eeg_val, X_eeg_test, y_train, y_val, y_test
        gc.collect()
        torch.cuda.empty_cache()
        
        # Create data loaders with modified parameters
        train_loader = DataLoader(
            train_dataset, batch_size=batch_size, shuffle=True, 
            pin_memory=True, num_workers=0  # Changed from 2 to 0
        )

        val_loader = DataLoader(
            val_dataset, batch_size=batch_size, shuffle=False,
            pin_memory=True, num_workers=0  # Changed from 2 to 0
        )

        test_loader = DataLoader(
            test_dataset, batch_size=batch_size, shuffle=False,
            pin_memory=True, num_workers=0  # Changed from 2 to 0
        )

        # Initialize model
        emg_input_dim = 8  # Number of EMG channels
        eeg_input_dim = 8  # Number of EEG channels
        
        model = MultimodalNet(
            emg_input_dim=emg_input_dim,
            eeg_input_dim=eeg_input_dim,
            hidden_dim=hidden_dim,
            num_classes=num_classes
        ).to(device)
        
        # Count parameters
        total_params = sum(p.numel() for p in model.parameters())
        print(f"Model has {total_params:,} parameters")
        
        # Define optimizer and loss function
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)
        
        # Define cosine annealing learning rate scheduler
        scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=learning_rate/100)
        
        # Train the model
        trained_model, history = train_model(
            model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=num_epochs
        )
        
        print("Training completed!")
        
        # WORKAROUND: Skip loading the saved checkpoint and use the just-trained model instead
        print("Checkpoint loading failed - using the most recently trained model instead.")
        
        # Use the metrics from the last epoch of training
        if history and 'val_f1' in history and len(history['val_f1']) > 0:
            best_epoch = num_epochs - 1  # Last epoch
            best_val_f1 = history['val_f1'][-1]
            best_val_acc = history['val_acc'][-1]
            
            print(f"Using model from final epoch {best_epoch+1} with validation F1: {best_val_f1:.4f}, "
                  f"validation accuracy: {best_val_acc:.2f}%")
        else:
            # Fallback if history isn't available
            best_epoch = num_epochs - 1
            best_val_f1 = 0.0
            best_val_acc = 0.0
            print("Using model from final epoch (metrics not available)")
        
        # Evaluate the model on test set
        test_results = evaluate_model(model, test_loader)
        
        print(f"Test F1 Score: {test_results['f1']:.4f}")
        print(f"Test Accuracy: {test_results['accuracy']:.2f}%")
        
        # Save final results
        final_results = {
            'best_epoch': best_epoch,
            'best_val_f1': best_val_f1,
            'best_val_acc': best_val_acc,
            'test_results': test_results,
            'history': history
        }
        
        # Save as numpy file for later analysis
        np.save('model_results.npy', final_results)
        
    except Exception as e:
        print(f"Error occurred: {e}")
        import traceback
        traceback.print_exc()
if __name__ == "__main__":
    main()

Using device: cuda
Loading data...
Normalizing data...
Found 378 common samples between EMG and EEG data.
Created 19576 windows from 378 samples.
Number of classes: 7
EMG windows shape: (19576, 50, 8)
EEG windows shape: (19576, 50, 8)
Training set: 11752 samples
Validation set: 3900 samples
Test set: 3924 samples
Model has 318,343 parameters


C:\Users\work\AppData\Local\Temp\ipykernel_5380\2281970095.py:257: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
C:\Users\work\AppData\Local\Temp\ipykernel_5380\2281970095.py:285: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/30, Train Loss: 1.0230, Train Acc: 62.89%, Train F1: 0.6263, Val Loss: 0.9969, Val Acc: 62.03%, Val F1: 0.6154
Precision: 0.7088, Recall: 0.6203
Saved model with F1: 0.6154, Accuracy: 62.03%


C:\Users\work\AppData\Local\Temp\ipykernel_5380\2281970095.py:285: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 2/30, Train Loss: 0.5979, Train Acc: 79.34%, Train F1: 0.7923, Val Loss: 0.8132, Val Acc: 71.31%, Val F1: 0.7050
Precision: 0.7710, Recall: 0.7131
Saved model with F1: 0.7050, Accuracy: 71.31%


...

Epoch 6/30, Train Loss: 0.2594, Train Acc: 91.36%, Train F1: 0.9132, Val Loss: 0.5448, Val Acc: 83.10%, Val F1: 0.8291
Precision: 0.8527, Recall: 0.8310
Saved model with F1: 0.8291, Accuracy: 83.10%


C:\Users\work\AppData\Local\Temp\ipykernel_5380\2281970095.py:285: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 7/30, Train Loss: 0.2304, Train Acc: 92.07%, Train F1: 0.9204, Val Loss: 0.5196, Val Acc: 83.64%, Val F1: 0.8316
Precision: 0.8619, Recall: 0.8364
Saved model with F1: 0.8316, Accuracy: 83.64%


C:\Users\work\AppData\Local\Temp\ipykernel_5380\2281970095.py:285: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


KeyboardInterrupt: 